# Feature Detection

In [1]:
# import methods from FIA python script
from FIA import *
import os

# Ignore seaborn warning for future deprecation of module part
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Library Preparation

In [6]:
experiment_path = "../data/example data/exampleA_ecolistrains/mg1655_P3-A8_neg.mzXML"
experiment = read_experiment(experiment_path)

In [7]:
centroid_exp = centroid_experiment(experiment, deepcopy=True)
oms.MzMLFile().store("../runs/centroided_experiment.MzML", centroid_exp)

'a/b/callable'

In [ ]:
# merge_exp = merge_spectra(centroid_exp, block_size=centroid_exp.getNrSpectra(), deepcopy=True)

## Untargeted

filepaths = ["../runs/tr_example/centroided_experiment.MzML"]
fms = [ untargeted_feature_detection(filepath=filepath,
                                     feature_filepath="../runs/tr_example/untargeted.featureXML",
                                     mass_error_ppm=10.0,
                                     noise_threshold_int=10000.0,
                                     remove_single_traces="true") 
        for filepath in filepaths ]
fms_rt = align_retention_times(fms)
fms_a = detect_adducts(fms_rt)

In [ ]:
store_feature_maps(fms_a)

## Targeted

#### From mass

In [ ]:
# set path to your mzML files and workfolder
data_dir = "../data/example data/exampleA"
run_dir = "../run/exampleA"

data_dir = os.path.normpath(os.path.join(os.getcwd(), data_dir))
run_dir = os.path.normpath(os.path.join(os.getcwd(), run_dir))

clean_dir(run_dir)

In [ ]:
run_dir = centroid_batch(data_dir, run_dir, file_ending=".mzML")

In [ ]:
feature_maps = untargeted_features_detection(run_dir, ".mzML", 
                                            mass_error_ppm=10.0, noise_threshold_int=1000.0,
                                            width_filtering="fixed", isotope_filtering_model="none", remove_single_traces="true",
                                            mz_scoring_by_elements="false", report_convex_hulls="true")

In [ ]:
feature_maps = align_retention_times(feature_maps, 
                                    max_num_peaks_considered=-1,      # -1 -> consider infinite/all peaks
                                    max_mz_difference=10.0,
                                    mz_unit="ppm")
plot_feature_map_rt_alignment(feature_maps)
consensus_map = consensus_features_linking(feature_maps, feature_grouper="KD")
mass_search_df = accurate_mass_search(consensus_map, "../runs/tr1", "tmp",
 "PositiveAdducts.tsv", "NegativeAdducts.tsv", "HMDBMappingFile.tsv", "HMDB2StructMapping.tsv", ionization_mode="negative")
cm_df = consensus_map_to_df(consensus_map)
filtered_cm_df = filter_consensus_map_df(cm_df, max_missing_values=1, min_feature_quality=0.8)
imputed_cm_df = impute_consensus_map_df(filtered_cm_df, n_nearest_neighbours=2)
def consensus_map_df_to_id_df(consensus_map_df:pd.DataFrame, mass_search_df:pd.DataFrame, result_dir:str) -> pd.DataFrame:
    clean_dir(result_dir)
    id_df = consensus_map_df

    id_df["identifications"] = pd.Series(["" for x in range(len(id_df.index))])

    for rt, mz, description in zip(
        mass_search_df["retention_time"],
        mass_search_df["exp_mass_to_charge"],
        mass_search_df["description"],
    ):
        indices = id_df.index[
            isclose(id_df["mz"], float(mz), atol=1e-05)
            & isclose(id_df["RT"], float(rt), atol=1e-05)
        ].tolist()
        for index in indices:
            if description != "null":
                id_df.loc[index, "identifications"] += str(description) + ";"
    id_df["identifications"] = [
        item[:-1] if ";" in item else "" for item in id_df["identifications"]
    ]
    id_df.to_csv(os.path.join(result_dir, "result.tsv"), sep="\t", index=False)
    return id_df
id_df = consensus_map_df_to_id_df(imputed_cm_df, mass_search_df, "../runs/tr1/results")
plot_id_df(id_df)

#### From mass, charge & retention time

In [ ]:
mt = define_metabolite_table("../databases/compounds.tsv", [50.0, 3000.0])
len(mt)

In [ ]:
fm = feature_detection_targeted("../runs/tr_example/centroided_experiment.MzML", mt[0:100], mz_window=5.0, rt_window=200.0, peak_width=3.0)

In [ ]:
oms.FeatureXMLFile().store("../runs/tr_example/targeted.featureXML", fm)